In [1]:
from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.metrics import balanced_accuracy_score

In [2]:
# Load data 
outliers_removed_df = pd.read_csv('indicators_bmi_filtered.csv')
outliers_removed_df.head()

,DIABETE4,_RACE,TOLDHI3,BPHIGH6,_BMI5,SMOKE100,_RFBING5,EDUCA,GENHLTH,_AGEG5YR,EXERANY2,FRUIT2,VEGETAB2,_INCOMG1,MEDCOST1,_SEX
0,No,White,Yes,No,14.54,Yes,Yes,High School Grad,Poor,Age 70 to 74,No,Yes,Yes,"$25,000 to < $35,000",No,Female
1,Yes,Black,No,Yes,28.29,No,Yes,High School Grad,Very Good,Age 70 to 74,No,Yes,Yes,"$15,000 to < $25,000",No,Female
2,Yes,White,Yes,Yes,33.47,No,No,High School Grad,Very Good,Age 60 to 64,Yes,Yes,Yes,"$50,000 to < $100,000",No,Female
3,Yes,Multiracial,Yes,No,28.73,No,Yes,Some High School,Poor,Age 75 to 79,Yes,Yes,Yes,"$15,000 to < $25,000",No,Male
4,No,White,No,No,24.37,Yes,Yes,Some College,Good,Age 80 or older,No,Yes,Yes,"$35,000 to < $50,000",No,Male


In [3]:
outliers_removed_df.dtypes

DIABETE4     object
_RACE        object
TOLDHI3      object
BPHIGH6      object
_BMI5       float64
SMOKE100     object
_RFBING5     object
EDUCA        object
GENHLTH      object
_AGEG5YR     object
EXERANY2     object
FRUIT2       object
VEGETAB2     object
_INCOMG1     object
MEDCOST1     object
_SEX         object
dtype: object

In [4]:
# Generate our categorical variable list
indicators_cat = outliers_removed_df.dtypes[outliers_removed_df.dtypes == "object"].index.tolist()

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False, drop='first')

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(outliers_removed_df[indicators_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names_out(indicators_cat)
encode_df.head()

,DIABETE4_Yes,_RACE_American Indian or Alaskan Native,_RACE_Asian,_RACE_Black,_RACE_Hispanic,_RACE_Multiracial,_RACE_Other,_RACE_White,TOLDHI3_Yes,BPHIGH6_Yes,...,FRUIT2_Yes,VEGETAB2_Yes,"_INCOMG1_$100,000 to < $200,000","_INCOMG1_$15,000 to < $25,000","_INCOMG1_$25,000 to < $35,000","_INCOMG1_$35,000 to < $50,000","_INCOMG1_$50,000 to < $100,000","_INCOMG1_Less than $15,000",MEDCOST1_Yes,_SEX_Male
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [6]:
# Merge one-hot encoded features and drop the originals
outliers_removed_df = outliers_removed_df.merge(encode_df,left_index=True, right_index=True)
outliers_removed_df = outliers_removed_df.drop(indicators_cat,1)
outliers_removed_df.head()

C:\Users\ashle\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,_BMI5,DIABETE4_Yes,_RACE_American Indian or Alaskan Native,_RACE_Asian,_RACE_Black,_RACE_Hispanic,_RACE_Multiracial,_RACE_Other,_RACE_White,TOLDHI3_Yes,...,FRUIT2_Yes,VEGETAB2_Yes,"_INCOMG1_$100,000 to < $200,000","_INCOMG1_$15,000 to < $25,000","_INCOMG1_$25,000 to < $35,000","_INCOMG1_$35,000 to < $50,000","_INCOMG1_$50,000 to < $100,000","_INCOMG1_Less than $15,000",MEDCOST1_Yes,_SEX_Male
0,14.54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,28.29,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,33.47,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,28.73,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,24.37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [7]:
# Define features and targets
y = outliers_removed_df['DIABETE4_Yes']
X = outliers_removed_df.drop(columns=['DIABETE4_Yes'])

In [8]:
# Split into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 78, stratify=y)
Counter(y_train)

Counter({0.0: 155232, 1.0: 26869})

In [9]:
# Create Standard Scaler
scaler = StandardScaler()

In [10]:
# Fit Data
X_scaler = scaler.fit(X_train)

# Scale Data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Undersample the data using `RandomUnderSampler`
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({0.0: 26869, 1.0: 26869})

# Logistic Regression 

In [12]:
# Fit a Logistic regression model using random undersampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
clf = model.fit(X_resampled, y_resampled)

In [13]:
# Validate and make predictions with testing data 
predictions = model.predict(X_test_scaled)

In [14]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index=['Actual 0', 'Acutal 1'], columns = ['Predicted 0', 'Predicted 1'])
cm_df

,Predicted 0,Predicted 1
Actual 0,35933,15811
Acutal 1,2325,6632


In [15]:
# Calculate the Balanced Accuracy Score
balanced_accuracy_score(y_test, predictions)

0.7174322422773853

In [16]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.94      0.69      0.74      0.80      0.72      0.51     51744
        1.0       0.30      0.74      0.69      0.42      0.72      0.52      8957

avg / total       0.84      0.70      0.73      0.74      0.72      0.51     60701



In [17]:
print(clf.coef_)

[[ 0.36396071  0.01344445  0.02251775 -0.06127893 -0.04457503 -0.05358392
  -0.04442601 -0.22774558  0.25003595  0.34254296 -0.02179477  0.17949341
   0.02507334  0.02289114  0.02470139  0.05190756  0.02321734  0.50706043
   0.50310019  0.30733529  0.26046506  0.09523261  0.17397882  0.25806168
   0.33514838  0.38645011  0.46039988  0.54706168  0.58304155  0.63210242
   0.64833864  0.54638552  0.51423547 -0.08452368  0.00370243 -0.01214167
   0.0324317   0.09083554  0.09779471  0.0749269   0.07807493  0.0774585
  -0.00974815  0.04595846]]


In [34]:
coef = pd.DataFrame(zip(X_train, np.transpose(clf.coef_.tolist()[0])), columns=['features', 'coef'])
coef

,features,coef
0,_BMI5,0.363961
1,_RACE_American Indian or Alaskan Native,0.013444
2,_RACE_Asian,0.022518
3,_RACE_Black,-0.061279
4,_RACE_Hispanic,-0.044575
5,_RACE_Multiracial,-0.053584
6,_RACE_Other,-0.044426
7,_RACE_White,-0.227746
8,TOLDHI3_Yes,0.250036
9,BPHIGH6_Yes,0.342543


In [35]:
coef.sort_values('coef')

,features,coef
7,_RACE_White,-0.227746
33,EXERANY2_Yes,-0.084524
3,_RACE_Black,-0.061279
5,_RACE_Multiracial,-0.053584
4,_RACE_Hispanic,-0.044575
6,_RACE_Other,-0.044426
10,SMOKE100_Yes,-0.021795
35,VEGETAB2_Yes,-0.012142
42,MEDCOST1_Yes,-0.009748
34,FRUIT2_Yes,0.003702


In [20]:
# p value
import statsmodels.api as sm
logit_model=sm.Logit(y_train, X_train)
result=logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.351847
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           DIABETE4_Yes   No. Observations:               182101
Model:                          Logit   Df Residuals:                   182057
Method:                           MLE   Df Model:                           43
Date:                Thu, 20 Apr 2023   Pseudo R-squ.:                  0.1591
Time:                        14:01:42   Log-Likelihood:                -64072.
converged:                       True   LL-Null:                       -76198.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
_BMI5                                       0.0471

# Random Forest

In [21]:
# Create Random Forest Classifier 
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

In [22]:
# Fit Model
rf_model = rf_model.fit(X_resampled, y_resampled)

In [23]:
# Validate and make predictions with testing data 
predictions = rf_model.predict(X_test_scaled)

In [24]:
# Evaluate performance with accuracy score
balanced_accuracy_score(y_test, predictions)

0.6875072129646564

In [25]:
# Generate confusion matrix
cm = confusion_matrix(y_test, predictions)

In [26]:
# Create df from confusion matrix
cm_df = pd.DataFrame(cm, index=['Actual 0', 'Acutal 1'], columns = ['Predicted 0', 'Predicted 1'])
cm_df

,Predicted 0,Predicted 1
Actual 0,35170,16574
Acutal 1,2729,6228


In [27]:
# Classification Report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.93      0.68      0.70      0.78      0.69      0.47     51744
        1.0       0.27      0.70      0.68      0.39      0.69      0.47      8957

avg / total       0.83      0.68      0.69      0.73      0.69      0.47     60701



In [28]:
# Rank importance of features
importances = rf_model.feature_importances_
importances

array([0.32054784, 0.00447803, 0.00460324, 0.01009947, 0.00934007,
       0.00503888, 0.00213111, 0.01640686, 0.0438786 , 0.07588971,
       0.03518095, 0.01579448, 0.00372365, 0.02542501, 0.00035883,
       0.0295053 , 0.00768643, 0.02692986, 0.02115811, 0.01145382,
       0.02107782, 0.00618405, 0.00759435, 0.0081459 , 0.00914547,
       0.01013973, 0.01178019, 0.01304063, 0.01437051, 0.01409756,
       0.01329559, 0.01179056, 0.01170228, 0.02540447, 0.00897345,
       0.00711504, 0.01471263, 0.01274941, 0.01450183, 0.01631136,
       0.0198049 , 0.00944702, 0.01267648, 0.0363085 ])

In [29]:
# Sort features 
sorted(zip(rf_model.feature_importances_,X.columns), reverse=True)

[(0.32054783791210967, '_BMI5'),
 (0.07588971023697058, 'BPHIGH6_Yes'),
 (0.04387859963718618, 'TOLDHI3_Yes'),
 (0.03630850161936478, '_SEX_Male'),
 (0.0351809482012571, 'SMOKE100_Yes'),
 (0.029505298661066822, 'EDUCA_Some College'),
 (0.02692986200369235, 'GENHLTH_Fair'),
 (0.02542500945992977, 'EDUCA_High School Grad'),
 (0.025404471421472757, 'EXERANY2_Yes'),
 (0.021158107616040325, 'GENHLTH_Good'),
 (0.021077821654909228, 'GENHLTH_Very Good'),
 (0.019804903870386543, '_INCOMG1_$50,000 to < $100,000'),
 (0.016406864614191687, '_RACE_White'),
 (0.016311361193072922, '_INCOMG1_$35,000 to < $50,000'),
 (0.01579447946190817, '_RFBING5_Yes'),
 (0.01471262782562731, '_INCOMG1_$100,000 to < $200,000'),
 (0.014501830880754166, '_INCOMG1_$25,000 to < $35,000'),
 (0.014370509379405401, '_AGEG5YR_Age 60 to 64 '),
 (0.014097563908002078, '_AGEG5YR_Age 65 to 69 '),
 (0.013295589541459011, '_AGEG5YR_Age 70 to 74 '),
 (0.013040630326948787, '_AGEG5YR_Age 55 to 59 '),
 (0.012749408683966595, '_INCO